In [1]:
:opt no-lint

# The World is a ~~Monad~~ Burrito
## CS 452, Spring 2022

# Introduction to Haskell

- Pure functional language
- Elm is a spiritual subset of Haskell built for the frontend
- Subset of functionality, slightly different syntax
    - Then a bunch of browser stuff bolted on
- Haskell named after Haskell *Curry*

### Difference: Haskell is Lazy

- Data is not evaluated until it is needed

In [3]:
2 + 2

4

In [40]:
-- First three items of a finite list
take 3 [1,2,3,4,5]

[1,2,3]

In [41]:
-- First three items of an infinite list
take 3 [1..]

[1,2,3]

In [39]:
take

head :: forall a. [a] -> a

In [4]:
-- Functions can be split across multiple lines
fact :: Integer -> Integer
fact 0 = 1
fact n = n * fact (n - 1)

fact 10

3628800

## Map

- Virtually identical to Elm

In [22]:
map (*2) [1,2,3]

[2,4,6]

## Folds

- Associate either to the right (fold right) or left (fold left)
- For lists, thing of it as replacing cons with a function
- Requires a starting value (what to use for empty list)

In [5]:
foldl (+) 0 [1,2,3]

7

In [9]:
-- Functions that commute will have identical foldl and foldr
foldr (+) 0 [1,2,3]

6

In [12]:
-- ((1 / 2) / 3) / 1)
foldl (/) 1 [1,2,3]


-- 1 / (2 / (3 / 1))
foldr (/) 1 [1,2,3]

0.16666666666666666

1.5

## Types and Classes

- Nearly identical to Elm
- Replace `type` keyword with `data`

In [18]:
data MyBool = MyFalse | MyTrue
:t MyFalse

MyFalse :: MyBool

In [21]:
type Pos = (Int, Int)
data Move = North | South | East | West

move :: Move -> Pos -> Pos
move North (x,y) = (x, y+1)
move South (x,y) = (x, y-1)
move East (x,y) = (x+1, y)
move West (x,y) = (x-1, y)

move West (2,2)

(1,2)

In [53]:
North

North

In [47]:
class Stringable a where
    stringify :: a -> String
    
instance Stringable Move where
    stringify North = "North"
    stringify South = "North"
    stringify East = "North"
    stringify West = "North"

In [48]:
:t stringify 

stringify :: forall a. Stringable a => a -> String

In [49]:
stringify North

"North"

In [51]:
-- Builtin version of Stringable above, called Show
instance Show Move where
    show North = "North"
    show South = "North"
    show East = "North"
    show West = "North"

In [52]:
North

North

## The Maybe type

- Same as the Maybe type we saw in Elm

```haskell
data Maybe a = Nothing | Just a
```

In [2]:
Just 2

Just 2

- What happens if we want to apply a function to something wrapped in a Maybe?

![ouch](https://adit.io/imgs/functors/no_fmap_ouch.png)

In [3]:
Just 2 + 3

: 

# Functors

- Functors are type classes
- Functors define `fmap` which determines how to apply a function to these types

$$
\text{fmap} :: (a \rightarrow b) \rightarrow f a \rightarrow f b
$$

In [4]:
fmap (+3) (Just 2)

Just 5

![fmap in action](https://adit.io/imgs/functors/fmap_just.png)

In [5]:
fmap (+3) Nothing

Nothing

Applying functions in ~~inferior~~ other languages

```
post = Post.find_by_id(1)
if post
  return post.title
else
  return nil
end
```

Haskell:

```haskell
fmap (getPostTitle) (findPost 1)
```

- `<$>` is the infix version of fmap
- Lists are functors
    - For lists `fmap` = `map`

In [6]:
(+3) <$> [1,2,3]

[4,5,6]

# Applicatives

- We can use `fmap` to a apply a function to a functor, i.e. a function to something wrapped in a box
- Applicatives apply a wrapped function to a wrapped valued

![applicative boxes](https://adit.io/imgs/functors/applicative_just.png)

## Handling multiple arguments

- Functors only work with functions that take a single argument
- Applicatives allow for the application of functions with multiple arguments

One option would be to declare different fmaps for different arities (in fact this is what Elm does for List.map)

```haskell
fmap0 :: a -> f a
fmap1 :: (a -> b) -> f a -> f b
fmap2 :: (a -> b -> c) -> f a -> f b -> f c
..
```

Hmm.. that seems annoying. Currying to the rescue!

- Apply a function wrapped in a box to a value wrapped in a box
- Define two operations
   
1. pure tells us how to put something in a box
2. `<*>` tells us how to apply a function in a box to something in a box.


```haskell
pure :: a -> f a
<*> :: f (a -> b) -> f a -> f b
```


In [7]:
pure (+) <*> Just 1 <*> Just 2

Just 3

In [8]:
:t (+)
:t pure (+)
:t pure (+) <*> Just 2
pure (+) <*> Just 2 <*> Just 3
pure (+) <*> Nothing <*> Just 3

(+) :: forall a. Num a => a -> a -> a

pure (+) :: forall (f :: * -> *) a. (Applicative f, Num a) => f (a -> a -> a)

pure (+) <*> Just 2 :: forall a. Num a => Maybe (a -> a)

Just 5

Nothing

- The default apply operator for lists represents non-deterministic computation
- That is lists represent list of all possible results from a function

In [9]:
pure (+) <*> [1,2,3] <*> [4,5,6]

[5,6,7,6,7,8,7,8,9]

- We could also represent parallel computation over lists.
    - Can be built straight from applicatives
    - Already done for us with zip functions

In [54]:
-- Only works for functions with two arguments
zipWith (+) [1,2,3] [4,5,6]

[5,7,9]

In [75]:
zipped = pure (+) <*> ZipList [1,2,3] <*> ZipList [4,5,6]
zipped

ZipList {getZipList = [5,7,9]}

In [76]:
getZipList zipped

[5,7,9]

- Applicatives allow programming in a pure language with *effects*
- Allows functions to be created where the arguments are no longer just plain values, but may also have effects (such as the possibility of failure), having many wasy to succeed, or performing I/O

# Monads

http://james-iry.blogspot.com/2009/05/brief-incomplete-and-mostly-wrong.html

http://tpetricek.github.io/Talks/2018/monads-what-we-talk-about/nice/index.html#/

![monads](http://tpetricek.github.io/Talks/2018/monads-what-we-talk-about/nice/images/monads.jpg)

- Functors apply a function to a wrapped value
- Applicatives apply a wrapped function to a wrapped value
- Monads apply a function that returns a wrapped value to a wrapped value

In [10]:
half x =
   if even x then
       Just (x `div` 2)
    else
        Nothing
        
half 4

Just 2

In [11]:
half Nothing

: 

In [12]:
Just 3 >>= half
Just 4 >>= half

Nothing

Just 2

```haskell
class Monad m where
    (>>=) :: m a -> (a -> m b) -> m b
```

![the bind operator](https://adit.io/imgs/functors/bind_def.png)

![wrapping and unwrapping](https://adit.io/imgs/functors/monad_just.png)

In [13]:
Just 32 >>= half >>= half >>= half

Just 4

In [14]:
getLine

"hello"

In [17]:
-- getLine doesn't just return a string
-- it returns a String wrapped in IO
-- IO is a Monad
:t getLine

getLine :: IO String

In [19]:
:t putStrLn

putStrLn :: String -> IO ()

In [18]:
putStrLn "Hello, world!"

hello

In [62]:
putStrLn "What is your name?" >>= \_ -> getLine >>= \s -> putStrLn ("Hello " ++ s)

What is your name?
Hello defreez

In [ ]:
-- do blocks are syntactic sugar for repeated
-- binds with lambdas
do
    putStrLn "What is your name?"
    s <- getLine
    putStrLn s

What is your name?

In [23]:
:t putStrLn

putStrLn :: String -> IO ()

In [24]:
:t (>>=)

(>>=) :: forall (m :: * -> *) a b. Monad m => m a -> (a -> m b) -> m b

In [34]:
putStrLn "What is your name?" >>= \_ -> getLine >>= \name -> putStrLn ("Hello " ++ name)

What is your name?
Hello Daniel.

![monads are burritos](https://chrisdone.com/images/comics/monads_are_burritos.png)

# References

https://chrisdone.com/posts/monads-are-burritos/

http://learnyouahaskell.com/functors-applicative-functors-and-monoids

https://chrisdone.com/posts/monads-are-burritos/

http://tpetricek.github.io/Talks/2018/monads-what-we-talk-about/nice/index.html#/